### 用户流失预测分析与应用

#### 1 概述

- 本示例的基本场景是：业务部门希望数据部门能对流失用户做分析，找到流失用户的典型特征
  - 到底流失客户的哪些特性最显著
  - 当客户在哪些特征的什么条件下比较容易发生流失行为
  - 业务部门就能针对这些客户的典型特征做有针对性的业务优化及挽回动作。

- 本次数据工作的特点如下：
  - 关于特征提取的分析工作，目标交付物是特征重要性和特征规则
  - 该需求可通过决策树类的分类算法实现，并且决策树是最好的解释规则的算法
  - 业务部门需要了解规则的关系，那么必须提供规则图
  - 数据集大概率会出现样本不均衡问题，原因是流失用户一定是少量的

- 经过梳理后，决定使用XGBoost的分类算法来实现规则提取，集成树算法具有非常高的精度及鲁棒性，因此在数据分析工作（及竞赛等领域）中经常使用，XGBoost能直接提供规则图的展示

#### 2 代码

- **导入模块**

  ```python
  # 导入库
  import pandas as pd
  from sklearn.model_selection import train_test_split  # 数据分区库
  import xgboost as xgb
  from sklearn.metrics import accuracy_score, auc, confusion_matrix, f1_score, \
      precision_score, recall_score, roc_curve  # 导入指标库
  from imblearn.over_sampling import SMOTE  # 过抽样处理库SMOTE
  import matplotlib.pyplot as plt
  ```

  - 本示例中使用了以下库：

    - 用Pandas做数据读取和基本处理
    - 用sklearn的train_test_split实现切分数据集为训练集和预测集
    - XGBoost的分类算法用于模型训练和规则提取
    - 用sklearn.metrics中的多个评估指标评估XGboost模型的效果
    - imblearn.over_sampling中的SMOTE库做样本均衡处理
    - 用Matplotlib做图形输出，该库实际上是配合使用。
    - GraphViz本身是一个非常强大的、矢量图输出的第三方程序，它也为Python提供图形接口。它会用来与XGBoost的to_graphviz方法配合实现树形图输出

    ```
    pydot则是XGboost在展示树形图时会用到的。安装：
    1）安装GraphViz程序。登录http://www.graphviz.org/download/
    2）使用pip install pydot命令安装pydot包。
    3）将GraphViz的可执行程序的路径配置到系统路径的PATH中。这样Python在执行时才能通过系统路径找到GraphViz的可执行程序，例如默认安装位置C：\Program Files(x86)\Graphviz2.38\bin。
    ```

- **数据准备**

  ```python
  # 读取准备
  raw_data = pd.read_csv('classification.csv', delimiter=',')  # 读取数据文件
  X,y = raw_data.iloc[:, :-1],raw_data.iloc[:, -1]  # 分割X,y
  # 数据基本审查
  n_samples, n_features = X.shape  # 总样本量,总特征数
  print('samples: {0}| features: {1} | na count: {2}'.format(n_samples, n_features,raw_data.isnull().any().count()))
  ```

  ![](img\feature.webp)

  - 这里仅查看了样本量、特征数量和NA值数量。XGBoost具有容忍性，不处理NA值，XGBoost仍然会有效地应对；但是后续做样本均衡时强制要求数据集不能有NA值

- 数据预处理
  ```python
  # 填充缺失值
  X = X.fillna(X.mean())
  # 样本均衡处理
  #'''
  model_smote = SMOTE()  # 建立SMOTE模型对象
  X, y = model_smote.fit_sample(X,y)  # 输入数据并作过抽样处理
  #'''
  ```

  - 填充缺失值和样本均衡
    - 填充缺失值使用的是均值填充法
    - 样本均衡策略用的是过抽样处理

- 数据集拆分

  ```python
  # 拆分数据集
  X = pd.DataFrame(X,columns=raw_data.columns[:-1])
  X_train, X_test, y_train, y_test = train_test_split(
      X, y, test_size=.3, random_state=0)  # 将数据分为训练集和测试集
  ```

- xgb训练分类器模型

  ```python
  param_dist = {'objective': 'binary:logistic', 'n_estimators': 10,
                'subsample': 0.8, 'max_depth': 10, 'n_jobs': -1}
  model_xgb = xgb.XGBClassifier(**param_dist)
  model_xgb.fit(X_train, y_train)
  pre_y = model_xgb.predict(X_test)
  ```

  - 参数说明
    - 定义了一个dict传参
    - 指定分类的对象是二元分类
    - estimator数量是10个
    - 每个estimator使用样本的比例是80%
    - 最大深度10
    - 并使用全部的CPU资源

- 第7部分效果评估-混淆矩阵

  ```python
  tn, fp, fn, tp = confusion_matrix(y_test, pre_y).ravel()  # 获得混淆矩阵
  pd.DataFrame([[tp,fn],[fp,tn]],columns=['prediction-0','prediction-1'],index=['actual-0','actual-1'])
  
  '''
  confusion matrix 
   +----------+--------------+--------------+
  |          | prediction-0 | prediction-1 |
  +----------+--------------+--------------+
  | actual-0 |     174      |      40      |
  | actual-1 |      45      |     177      |
  +----------+--------------+--------------+
  
  '''
  # 核心评估指标
  y_score = model_xgb.predict_proba(X_test)  # 获得决策树的预测概率
  fpr, tpr, _ = roc_curve(y_test, y_score[:, 1])  # ROC
  auc_s = auc(fpr, tpr)  # AUC
  scores = [round(i(y_test, pre_y),3 )for  i in (accuracy_score,precision_score,\
                                       recall_score,f1_score)]
  scores.insert(0,auc_s)
  pd.DataFrame([scores],columns=['auc', 'accuracy', 'precision', 'recall', 'f1'])
  
  '''
  	auc	      accuracy   precision	 recall	   f1
  	0.896849	0.826	 0.814	      0.836	  0.825
  '''
  ```

  - 先通过决策树模型对象的predict_proba方法获得决策树对每个样本点的预测概率
  - 预测概率在下面的ROC计算中用到
  - 概率信息可作为基于阈值调整分类结果输出的关键，如可自定义阈值来做进一步精细化分类类别控制
  - 计算roc_curve，然后通过列表生成式分别计算sklearn.metrics的accuracy_score、precision_score、recall_score、f1_score，分别得到准确率、精确度、召回率、F1得分值
    - auc：AUC（Area Under Curve），ROC曲线下的面积。ROC曲线一般位于y=x上方，因此AUC的取值范围一般在0.5和1之间。AUC越大，分类效果越好
    - accuracy：准确率（Accuracy）
    - precision：精确度（Precision），分类模型的预测结果中将正例预测为正例的比例
      - 公式为：P=TP/(TP+FP)
    - recall：召回率（Recall），分类模型的预测结果被正确预测为正例占总的正例的比例
      - 公式为：R=TP/(TP+FN)，取值范围[0，1]，值越大说明分类结果越准确
      - F1得分（F-score），准确度和召回率的调和均值
        - 公式为：F1=2×(P×R)/(P+R)，取值范围[0，1]，
  - 调参说明
    - 样本均衡的影响是最关键的，读者可注释掉样本均衡的模块，会发现总体Accuracy、Precision能下降到0.7左右，而f1得分能下降到0.5左右。
    - 填充缺失值对XGBoost的效果几乎没影响，可同时注释掉填充缺失值和样本均衡模块查看结果。注意：仅注释掉填充缺失值而保留样本均衡会报NA错误。
    - param_dist对模型本身的调整上，subsample影响较大，原因是直接决定了子模型的训练样本量；其次是max_depth，最大树深度；最后是n_estimitors的大小。

- 输出可视化结果

  ```python
  # 输出特征重要性
  xgb.plot_importance(model_xgb,height=0.5,importance_type='gain',max_num_features=10,xlabel='Gain Split',grid=False) 
  ```
  ![image-20191125144439212](img\importance.webp)

  - 这里一行代码直接通过plot_importance方法调用Matplotlib的接口展示图形(barh图像)，所以需要导入Matplotlib
    - model_xgb：树模型对象
    - height：条形图的高度
    - importance_type：决定了特征重要性如何计算可选方式包括“weight”“gain”“cover”
      - “weight”是特征在树中出现的次数
      - “gain”是使用该特征分裂的平均增益值
      - “cover”是作为分裂节点的覆盖的样本比例。
    - max_num_features：设置显示的排序后最大特征的数量。
    - xlabel：X轴文字
    - grid：值为False设置不显示网格线

- ```python
  # 输出树形规则图
  xgb.to_graphviz(model_xgb, num_trees=1, yes_color='#638e5e', no_color='#a40000') 
  ```
  - 通过一行代码输出树形规则图
    - model_xgb：树模型对象
    - num_trees：指定要输出树的index值
    - 在XGBoost的模型中，通过参数n_esti-mators指定了10个基模型（也就是10棵树），因此这里最多只能输出10个树形图，索引从0到9
    - yes_color：值为真（符合条件）的线条颜色
    - no_color：值为假（不符合条件）的线条颜色
  - 数据中，流失标记的是1，未流失标记的是0。对应到图中就是左侧线；否则为False。那么，我们就是沿着左侧线条一直寻找即可

```python
# 前N条规则对应的用户数据
rule_depth_1 = X_test['internet']<0.00284512946
rule_depth_2 = X_test['longten']<230.75
rule_depth_3 = X_test['total_orders']<2.97253799
rule_depth_4 = X_test['sex']<0.972537994
rule_depth_5 = X_test['wireten']<86.0607376
rule_list = [rule_depth_1,rule_depth_2,rule_depth_3,rule_depth_4,rule_depth_5]
rule_pd = [pd.DataFrame(i) for i in rule_list]
rule_pd_merge = pd.concat(rule_pd,axis=1)

# 遍历不同条件下用户的情况
for i in range(5):
    dyn_rules = rule_pd_merge.iloc[:,:i+1] # 取出top规则
    dyn_rules['is_true']=[all(i)==True for i in dyn_rules.values]# 得到都为true的record
    y_test_selected = y_test[dyn_rules['is_true']]
    y_pre_selected = y_score[dyn_rules['is_true']]
    y_pre_cal = y_pre_selected[:,1] >=0.5
    total_samples = len(y_pre_cal)
    is_churn = y_pre_cal.sum()
    churn_rate = float(is_churn)/total_samples
    # 计算样本比例
    print('total samples: {}'.format(total_samples))
    print('churn samples: {} | rate: {:.0%} '.format(is_churn,churn_rate))
    print('unchurn samples: {} | rate: {:.0%} '.format((total_samples-is_churn),(1-churn_rate)))
    print('-'*40)
    
    '''
    total samples: 209
churn samples: 37 | rate: 18% 
unchurn samples: 172 | rate: 82% 
----------------------------------------
total samples: 95
churn samples: 34 | rate: 36% 
unchurn samples: 61 | rate: 64% 
----------------------------------------
total samples: 32
churn samples: 17 | rate: 53% 
unchurn samples: 15 | rate: 47% 
----------------------------------------
total samples: 21
churn samples: 14 | rate: 67% 
unchurn samples: 7 | rate: 33% 
----------------------------------------
total samples: 19
churn samples: 13 | rate: 68% 
unchurn samples: 6 | rate: 32% 
----------------------------------------
    
    '''
```

- 结果解读

  - 从上层到下层，随着条件的增加（越多条件就越能精准地定位用户），流失用户的概率越来越高

  - 当规则越多时，覆盖的总样本量和流失用户的数量越少

  - 真正把这些规律讲给业务部门时，需要让业务人员综合样本量以及流失用户的概率综合判断

    - 如果条件过多，虽然选择的用户越精准，但却会无法覆盖更多的可能用户
    - 如果条件太少，又会导致用户过于泛泛，也无法达到精准的目的
    - 这些规则，最终给到业务方，业务方可直接在数据库中，通过SQL检索查询符合要求的客户，然后做有针对性的运营工作。

    

# 课程回顾

- 用户流失规则建模案例

  - 分类问题，正负样本不均衡如何处理

    - 过采样  通过重复复制数据少的类别所对应的样本，实现样本均衡

    - 当样本不均衡的时候 通过过采样可以有效的解决问题

      ```python
      from imblearn.over_sampling import SMOTE  # 过抽样处理库SMOTE
      model_smote = SMOTE()  # 建立SMOTE模型对象
      X, y = model_smote.fit_sample(X,y)  # 输入数据并作过抽样处理
      ```

  - xgboost   boosting的集成学习算法， 对数据的要求相对比较低，不需要做归一化，用缺失值也可以容忍

    ```python
    import xgboost as xgb
    param_dist = {'objective': 'binary:logistic', 'n_estimators': 10,
                  'subsample': 0.8, 'max_depth': 10, 'n_jobs': -1}
    model_xgb = xgb.XGBClassifier(**param_dist)
    model_xgb.fit(X_train, y_train)
    pre_y = model_xgb.predict(X_test)
    ```

  - 二分类问题评估

    - 是不是羊毛（是羊毛的概率）， 是否会流失，是否会逾期，用户是否会点击（是否会点击的概率）
    - auc   推荐系统 风控建模 是最终要的一个指标  一般 0.6以上就可以勉强用了 在推荐业务 风控业务中 auc 在0.7~0.8之间 如果能在0.8以上就很好了 很少有超过0.85的
    - 精准率 presicion rate    所有预测为true的样本中 真正为true的比例   投资领域 比较看重精准率
    - 召回率  recall rate   所有真实为true的样本中，被成功预测的比例   疾病检测
    - F1 score  精准率 召回率的调和平均值  只有精准率和召回率都高的时候，才可能大

- 在很多业务场景下，通过一个分类的模型，最终不是要预测结果，而是要找到影响结果最重要的因素
  - 在这样的场景下 算法的可解释性就特别的重要了
    - 逻辑回归
    - 随机森林  GBDT XGBoost  Lightbgm